In [1]:
import os
import boto3
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
access_key_id = os.getenv("Access_key_ID")
secret_access_key = os.getenv("Secret_access_key")
bucket_name = os.getenv("BUCKET_NAME")
region_name = os.getenv("REGION_NAME")

In [5]:
s3 = boto3.client('s3',
                  aws_access_key_id=access_key_id,
                  aws_secret_access_key=secret_access_key,
                  region_name=region_name)